In [ ]:
"""
IMPORT LIBRARIES
"""

import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import json
import os
import keras
from tensorflow.keras import activations
import copy
from tensorflow.python.client import device_lib
#tf.config.list_physical_devices("GPU")
device_lib.list_local_devices()

In [ ]:
"""
IMPORT SUBDATASET OF TRAINING, TESTING DATASET
"""
 
train = json.load(open(os.path.join(os.getcwd(), "dataset/train/notev7.json")))
val = json.load(open(os.path.join(os.getcwd(), "dataset/val/notev7.json")))
test = json.load(open(os.path.join(os.getcwd(), "dataset/test/notev7.json")))

trainStaff = json.load(open(os.path.join(os.getcwd(), "dataset/train/StaffAndSep.json")))
valStaff = json.load(open(os.path.join(os.getcwd(), "dataset/val/StaffAndSep.json")))
testStaff = json.load(open(os.path.join(os.getcwd(), "dataset/test/StaffAndSep.json")))



def extractData(datasetStaff, datasetNotes, dirn = None):
    mask = []
    TotalSortedList = []
    for data in datasetStaff:
        annotation = [a for a in datasetStaff[data]['regions']]
        sortedList = [i for i in range(len(annotation))]
        for k_index, a in enumerate(sortedList): 
            index  = k_index
            if index != len(sortedList) -1:
                while index < len(sortedList)-1:
                    if annotation[a]['shape_attributes']['y'] > annotation[sortedList[index+1]]["shape_attributes"]["y"]:
                        i = sortedList[index+1]
                        sortedList[index+1] = sortedList[index]
                        sortedList[index] = i
                        break
                    index += 1

            if index > 0:
                while index > 0:
                    if annotation[sortedList[index]]['shape_attributes']['y'] < annotation[sortedList[index-1]]["shape_attributes"]["y"]:
                        i = sortedList[index-1]
                        sortedList[index-1] = sortedList[index]
                        sortedList[index] = i
                    index -= 1

        TotalSortedList.append(sortedList)


    TrainDataset = []
    ValDataset = []
    c = ['G4','A4','B4','C3','D3','E3','F3','G3','A3','B3','C2','D2','E2','F2','G2','A2','B2','C','D','E','F','G']


    for image in datasetNotes:
        if len(datasetNotes[image]['regions']) > 0:
            _id = ''
            index = 0
            for i, a in enumerate(datasetStaff):
                if datasetNotes[image]['filename'] == datasetStaff[a]['filename']:
                    _id  = a
                    index = i
                    break

            img = cv2.imread(os.path.join(os.getcwd(), f"dataset/Notev1/{dirn}/{datasetNotes[image]['filename']}"))
            start, end = 0, img.shape[0]

            b = 0
            while b < len(TotalSortedList[index]):
                Height = datasetStaff[_id]['regions'][TotalSortedList[index][b]]['shape_attributes']['height']
                if b > 0: 
                    start = datasetStaff[_id]['regions'][TotalSortedList[index][b-1]]['shape_attributes']['y']

                if b + 1 < len(TotalSortedList[index]):
                    end = datasetStaff[_id]['regions'][TotalSortedList[index][b+1]]['shape_attributes']['y']


                
                for note in datasetNotes[image]['regions']:
                    load = False
                    if note["region_attributes"]['class_ids'] in c:
                        load = True


                    if load == True and note['shape_attributes']['y'] > start and note['shape_attributes']['y'] + note['shape_attributes']['height'] < end:
                        
                        
                        
                        x = note['shape_attributes']['x']
                        y = note['shape_attributes']['y']
                        w = note['shape_attributes']['width']
                        h = note['shape_attributes']['height']
                        sh = datasetStaff[_id]['regions'][TotalSortedList[index][b]]['shape_attributes']['height']
                        sy = datasetStaff[_id]['regions'][TotalSortedList[index][b]]['shape_attributes']['y']
                        
                        def distribute(data):
                            distri = []

                            av = np.average(data)
                            for i, r in enumerate(data):
                                distri.append(0)
                                for c in r:
                                    if  0.299*c[0] + 0.587*c[1] + 0.114*c[2] < av:

                                        distri[i] += 1

                            distri = np.array(distri)/data.shape[1]
                            
                            for i,v in enumerate(distri):
                                if v > 0.9:
                                    distri[i] = 0
                                else:
                                    distri[i] = 1
                                    
                            return distri
                        
                        
                       
                        endm = int(sy+2*Height)
                        startm = int(sy - Height)
                        if endm > (int(end)):
                            endm = int(end)
                        
                        if startm < start:
                            startm = start
                            
                        topmost = sy-startm
                        bottommost = y+h - startm
                        
                        if y-startm < topmost:
                            topmost = y - startm
                        
                        if bottommost < sy + sh -startm:
                            bottommost = sy + sh - startm
                        
                        
                        #mask = np.zeros((3*sh,w,6))
                            
                        mask = []
                        cmask =  copy.deepcopy(img[startm:endm,x:x+w,:])
                        staffFilter = distribute(img[sy:sy+sh,x:x+w,:])
            
                        
                        bmask = np.zeros((cmask.shape[0],cmask.shape[1],3)) + 255
                        bmask[sy-startm: sy+sh-startm,:,:] = 0
                        #mask.append(img[startm:endm,x:x+w,:])
                        
                        background = np.average(img[startm:endm,x:x+w,:]) + 255
                        background /= 2
                        
                        cmask[0:sy-startm,:,:] = background
                        cmask[sy+sh-startm: cmask.shape[0], :,:] = background
                        
                        bmask = copy.deepcopy(cmask)
                        
                        gooddata = True
                        
                        notem = np.zeros((h, w,3))
                        hrw = h/w
                        for nr in range(h):
                            for nc in range(w):
                                if (nr - h/2)**2 + (hrw)**2*(nc - w/2)**2 < h**2/4:
                                    notem[nr, nc,:] = 1
                                    
                        try:
                        
                            for mr, rv in enumerate(notem):
                                for mc,cv in enumerate(rv):
                                    if cv[0]== 0:
                                        cmask[y-startm+mr, mc,:] = 255
                                    else:
                                        cmask[y-startm+mr, mc,:] = copy.deepcopy(img[y+mr,x+mc,:])
                        except:
                            gooddata = False
                            pass
                        #print(h)
                        
                        #cmask[y - startm: y+h - startm,:,:] = copy.deepcopy(notem)
                        #print(h)
                                    
                                    
                        
                                
                                    
                            
                        #cmask = cv2.resize(cmask, (20,100))
                        #bmask = cv2.resize(bmask, (20,100))
                        
                        def finetuneImage(data, fil):
                            data = np.array(data)
                            fil = np.array(fil)
                            for i, v in enumerate(data):
                                for j,a in enumerate(v):
                                    data[i,j,0] *= fil[i]
                                    data[i,j,1] *= fil[i]
                                    data[i,j,2] *= fil[i]
                            
                            return data
                        
                        cmask[sy-startm: sy+sh-startm,:,:] = finetuneImage(cmask[sy-startm: sy+sh-startm,:,:], staffFilter)
                        
                        def distribute(data):
                            distri = []
                            av = np.average(data)
                            for i, r in enumerate(data):
                                distri.append(0)
                                for c in r:
                                    if  0.299*c[0] + 0.587*c[1] + 0.114*c[2] < av:

                                        distri[i] += 1
                                        
                            distri = np.array(distri)/data.shape[1]
                            
                            #pos = [1,0]

                            #for i,v in enumerate(distri):
                            #    if v > 0.8:
                            #        pos[0] = i
                            #        break
                            
                            #for i in range(len(distri)):
                            #    if distri[len(distri) - i-1] == 1:
                            #        pos[1]= len(distri) - i-1
                            #        break

                            return distri
                        
                        spos = distribute(cmask)
                        subsh = sh/4
                        rjk = spos
                        
                        
                        mask.append(cmask)
                        mask.append(copy.deepcopy(img[startm:endm,x:x+w,:]))
                      
                        
            
                        
                        
                        if gooddata == True:
                            TrainDataset.append({'mask': [cv2.resize(cmask, (20,100)),rjk], "class_ids": note["region_attributes"]['class_ids']})
                            
                            if dirn == 'train':
                                down = 5
                                while bottommost + down < mask[0].shape[0]:
                                    temcm = np.zeros(mask[0].shape) + 255
                                    temcm[topmost+down: bottommost+down,:,:] = copy.deepcopy(mask[0][topmost:bottommost,:,:])
                                    tembm = np.zeros(mask[1].shape) + 255
                                    tembm[topmost+down: bottommost+down,:,:] = copy.deepcopy(mask[1][topmost:bottommost,:,:])
                                    down += 5
                                    TrainDataset.append({'mask': [cv2.resize(temcm, (20,100)), rjk], "class_ids": note["region_attributes"]['class_ids']})
                                    expandDim = np.zeros((6,temcm.shape[1],3)) + 255
                                    #for k in range(10):
                                    #    temcm = np.concatenate((temcm,expandDim))
                                    #    TrainDataset.append({'mask': [cv2.resize(temcm, (20,100)), rjk], "class_ids": note["region_attributes"]['class_ids']})
                                    
                                down = 5
                                while topmost-down > 0:
                                    temcm = np.zeros(mask[0].shape) + 255
                                    temcm[topmost-down: bottommost-down,:,:] = copy.deepcopy(mask[0][topmost:bottommost,:,:])
                                    tembm = np.zeros(mask[1].shape) + 255
                                    tembm[topmost-down: bottommost-down,:,:] = copy.deepcopy(mask[1][topmost:bottommost,:,:])
                                    down += 5
                                    TrainDataset.append({'mask': [cv2.resize(temcm, (20,100)),rjk], "class_ids": note["region_attributes"]['class_ids']})
                                    expandDim = np.zeros((6,temcm.shape[1],3)) + 255
                                    #for k in range(10):
                                    #    temcm = np.concatenate((expandDim, temcm))
                                    #    TrainDataset.append({'mask': [cv2.resize(temcm, (20,100)), rjk], "class_ids": note["region_attributes"]['class_ids']})
                                
                            
                            



                b += 2    
    dict_c = {}
    for i, _class in enumerate(c):
        dict_c[_class] = i


    TotalMask = [[] for i in range(len(c))]
 
    

    
    for i in TrainDataset:
        TotalMask[dict_c[i['class_ids']]].append(i['mask'])
    
    return TotalMask

train_dataset = extractData(trainStaff,train, 'train')
val_dataset = extractData(valStaff, val, 'val')
test_dataset = extractData(testStaff, test, 'test')

testM_x = []

test_y = []

trainM_x = []

train_y = []

valM_x = []

val_y = []

for i,data in enumerate(train_dataset):
    for k, info in enumerate(data):
        trainM_x.append(info)
        
        train_y.append(i)
        
for i,data in enumerate(val_dataset):
    for k, info in enumerate(data):
        valM_x.append(info)
        
        val_y.append(i)
        
for i,data in enumerate(test_dataset):
    for k, info in enumerate(data):
        testM_x.append(info)
       
        test_y.append(i)
        

train_y = np.array(train_y)

trainM_x = np.array(trainM_x)


val_y = np.array(val_y)

valM_x = np.array(valM_x)


test_y = np.array(test_y)
  
testM_x = np.array(testM_x)


In [ ]:
"""
MERGE SUBDATASET
"""
def extract_infor(dataset):
    image1 = []
    for i, v in enumerate(dataset):
        image1.append(v[0])
        
    
    return np.array(image1)/255

trainM1_x = extract_infor(trainM_x)
testM1_x = extract_infor(testM_x)
valM1_x = extract_infor(valM_x)

In [ ]:
"""
INSPECT DATA
"""
j = 2000
#change j to view different images 

plt.imshow(trainM1_x[j])

In [ ]:
"""
BUILD THE PN_CNN MODEL

noteFocus(): apply position factor 
conv_block(): input goes through a convolutional layer before adding to final stage 
identity_block(): input is added directly to the final stage 

model's layers: input -> noteFocus -> CNN -> FCNN
"""


import keras
from tensorflow.keras import activations
from tensorflow.keras import layers
c = ['G4','A4','B4','C3','D3','E3','F3','G3','A3','B3','C2','D2','E2','F2','G2','A2','B2','C','D','E','F','G']

class noteFocus(keras.layers.Layer):
    def __init__(self):
        super(noteFocus, self).__init__()
    
    def build(self, input_shape):
        self.space = np.zeros((100, 20,3))
       
        for i in range(self.space.shape[0]):
            self.space[i,:,:] = i + 50
        self.space = self.space/150
        self.space = tf.convert_to_tensor(self.space, dtype = tf.float32)
        
        
        
    def get_w(self):
        return self.space2
        
    def call(self, inputs):
        r = tf.subtract(1.0,inputs)
        r = tf.multiply(r, self.space)
        r = tf.cast(r, dtype = tf.float32)
        return r





def conv_block(size, stride, input_tensor):
    x = keras.layers.Conv2D(size[0],(1, 1), padding = "same")(input_tensor)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation(activations.relu)(x)
    x = keras.layers.Conv2D(size[1],(stride, stride), padding = "same")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation(activations.relu)(x)
    x = keras.layers.Conv2D(size[2],(1, 1), padding = "same")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation(activations.relu)(x)
    
    shotcut = keras.layers.Conv2D(size[2],(1, 1), padding = "same")(input_tensor)
    shotcut = keras.layers.Activation(activations.relu)(shotcut)

    x = keras.layers.Add()([x, shotcut])
    x = keras.layers.Activation(activations.relu)(x)
    
    return x

def identity_block(size, strike, input_tensor):
    x = keras.layers.Conv2D(size[0],(1,1), padding = "same")(input_tensor)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation(activations.relu)(x)
    x = keras.layers.Conv2D(size[1],(3,3), padding = "same")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation(activations.relu)(x)
    x = keras.layers.Conv2D(size[2],(1,1), padding = "same")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation(activations.relu)(x)

    x = keras.layers.Add()([x, input_tensor])
    x = keras.layers.Activation(activations.relu)(x)
    
    return x





input_tensor = keras.layers.Input(shape = [100,20,3], name = "input")

#stage 1 
x = noteFocus()(input_tensor)
x = conv_block([64,64,64],1,input_tensor)
x = identity_block([64,64,64],3,x)

x = conv_block([64,64,64],3,input_tensor)
x = identity_block([64,64,64],3,x)
x = identity_block([64,64,64],3,x)
x = identity_block([64,64,64],3,x)

x = keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding="same")(x)

#stage 2
x = conv_block([128,128,128],3,x)
x = identity_block([128,128,128],3,x)
x = identity_block([128,128,128],3,x)
x = identity_block([128,128,128],3,x)
x = identity_block([128,128,128],3,x)

x = keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding="same")(x)

#stage 3
x = conv_block([256,256,256],3,x)

for i in range(4):
    x = identity_block([256,256,256],3,x)
x = keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding="same")(x)

#stage 4
x = conv_block([512,512,512],3,x)
for i in range(5):
    x = identity_block([512,512,512],3,x)

x = keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding="same")(x)


#stage 5
a = keras.layers.Flatten()(x)
a = keras.layers.Dense(8192, activation = 'relu')(a)
a = keras.layers.Dense(4096, activation = 'relu')(a)
a = keras.layers.Dense(2048, activation = 'relu')(a)
a = keras.layers.Dense(1024, activation = 'relu')(a)
a = keras.layers.Dense(512, activation = 'relu')(a)
a = keras.layers.Dense(256, activation = 'relu')(a)
a = keras.layers.Dense(128, activation = 'relu')(a)
a = keras.layers.Dense(96, activation = 'relu')(a)
a = keras.layers.Dense(48, activation = 'relu')(a)  


output = keras.layers.Dense(len(c), activation = "softmax")(a)


model  = keras.Model(inputs = [input_tensor], outputs = [output])

with tf.device('/CPU:0'):
    model.compile(loss="sparse_categorical_crossentropy", optimizer='sgd', metrics=['accuracy'])

In [ ]:
"""
IMAGE AUGMENTATION 
"""
from tensorflow.keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(zoom_range = [0.3,1.5])

In [ ]:
"""
TRAINING 
"""
import multiprocessing
with tf.device('/CPU:0'):
    his = model.fit_generator(aug.flow(trainM1_x, train_y), epochs = 5, steps_per_epoch =len(trainM2_x) , validation_steps = 300, validation_data=(valM1_x, val_y), shuffle=True, workers= multiprocessing.cpu_count())

In [ ]:
"""
TESTING
"""
prey = model.predict([testM1_x, testM2_x])
maxv = []
for ipred in range(len(prey)):
    
    maxs = 0
    for i,index in enumerate(prey[ipred]):
        if index > prey[ipred][maxs]:
            maxs = i

    maxv.append(maxs)

print(maxv)
c = ['G4','A4','B4','C3','D3','E3','F3','G3','A3','B3','C2','D2','E2','F2','G2','A2','B2','C','D','E','F','G']

In [ ]:
"""
GIVEN TESTING DATA
"""
print(test_y)